In [1]:
import cv2
import numpy as np
import itertools
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import imutils

In [42]:
def bottom_level(image, threshold=0.4):
    height, width, _ = image.shape
    bottle_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bottle_gray = cv2.GaussianBlur(bottle_gray, (7, 7), 0)
    # cv2_imshow(bottle_gray)
    # plt.hist(bottle_gray.ravel(), 256,[0, 256]); plt.show()
    (T, bottle_threshold) = cv2.threshold(bottle_gray, 55, 255, cv2.THRESH_BINARY_INV)
    # cv2_imshow(bottle_threshold)
    contours = cv2.findContours(bottle_threshold .copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    bottle_clone = image.copy()
    cv2.drawContours(bottle_clone, contours, -1, (255, 0, 0), 2)
    # cv2_imshow(bottle_clone)
    areas = [cv2.contourArea(contour) for contour in contours]
    (contours, areas) = zip(*sorted(zip(contours, areas), key=lambda a:a[1]))
    # print contour with largest area
    bottle_clone = image.copy()
    cv2.drawContours(bottle_clone, [contours[-1]], -1, (255, 0, 0), 2)
    # cv2_imshow(bottle_clone)
    bottle_clone = image.copy()
    (x, y, w, h) = cv2.boundingRect(contours[-1])
    # print(f'x{x} y{y} w{w} h{h}')
    # print(height)
    aspectRatio = h / float(height)
    # print(aspectRatio)
    if aspectRatio > threshold:
        cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(bottle_clone, "High", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
        # cv2_imshow(bottle_clone)
        return True
    else:
        cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(bottle_clone, "Low", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        # cv2_imshow(bottle_clone)
        return False
    # cv2_imshow(bottle_clone)
   

In [25]:
def white_detector(image, threshold=0.27):
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 70, 255])
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask_white = cv2.inRange(hsv_image, lower_white, upper_white)
    # cv2_imshow(mask_white)

    white_segment = cv2.bitwise_and(image, image, mask=mask_white)
    # cv2_imshow(white_segment)
    
    total_pixels = image.shape[0] * image.shape[1]
    white_pixels = np.sum(mask_white) // 255
    white_percentage = white_pixels / total_pixels
    # print(white_percentage)
    # print(f'Percentual de branco: {white_percentage}')
    
    return white_percentage >= threshold

In [26]:
def detect_misplaced_covers(image):
    # cv2_imshow(image)
    # Preprocess the image (grayscale, blur, etc.)
    height, width, _ = image.shape
    cover_line = image[:height//2, :]
    # cv2_imshow(cover_line)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    # cv2_imshow(blurred)

    threshold1 = 50
    threshold2 = 100
    rho = 1
    theta = np.pi/180
    threshold = 51

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, threshold1, threshold2)
    # cv2_imshow(edges)

    # Perform the Hough transform to detect lines
    lines = cv2.HoughLines(edges, rho, theta, threshold)
    green_lines = 0  # Counter for horizontal lines
    blue_lines = 0
    ignored_lines = 0  # Counter for line intersections
    perfectlyPlaced = False

    max_horizontal_threshold = 95*np.pi/180
    min_horizontal_threshold = 10*np.pi/180
    vertical_lines = []
    lineData = []
 
    if lines is not None:
        image_with_lines = image.copy()
        for line in lines:
            rho, theta = line[0]
            cos_theta = np.cos(theta)
            sin_theta = np.sin(theta)
            angle_diff = np.abs(theta - 0) #delete this
            x0 = cos_theta * rho
            y0 = sin_theta * rho
            pt1 = (int(x0 + 1000 * (-sin_theta)), int(y0 + 1000 * (cos_theta)))
            pt2 = (int(x0 - 1000 * (-sin_theta)), int(y0 - 1000 * (cos_theta)))
            # cv2.line(image_with_lines, pt1, pt2, (0, 255, 0), 2)
            if  (theta >= 0 and theta <=  3*np.pi/180):
              color = (0, 255, 255)
              ignored_lines += 1
              lineData.append([theta, 'ignorado'])
            elif (theta >= 178*np.pi/180 and theta <=  182*np.pi/180):
              color = (0, 255, 255)
              ignored_lines += 1
              lineData.append([theta, 'ignorado'])
            elif (theta < max_horizontal_threshold and theta > min_horizontal_threshold):
              lineData.append([theta, 'horizontal'])
              # print(theta, 'horizontal')
              color = (0, 255, 0)
              green_lines += 1
            else:
              lineData.append([theta, 'n passou'])
              color = (255, 0 , 0)
              vertical_lines.append(line)
              blue_lines += 1
              # cv2.line(image_with_lines, pt1, pt2, (0, 255, 0), 2)

            cv2.line(image_with_lines, pt1, pt2, color, 2)
        # cv2_imshow(image_with_lines)
        if green_lines >= blue_lines or ignored_lines == len(lines) or len(lines) == 0:
          perfectlyPlaced = True
        else:
            # cv2_imshow(blurred)
            # cv2_imshow(edges)
            # for data in lineData:
            #   print(data)
            # cv2_imshow(image_with_lines)
            perfectlyPlaced = False
            # print(perfectlyPlaced)
    # else:
    #   cv2_imshow(blurred)
    #   cv2_imshow(edges)
    # for data in lineData:
    #     print(data)
    # cv2_imshow(image_with_lines)
    # print(perfectlyPlaced)
    return perfectlyPlaced

In [27]:
def black_detector(image, threshold = 0.135):
    bottle_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bottle_gray = cv2.GaussianBlur(bottle_gray, (7, 7), 0)
    (T, bottle_threshold) = cv2.threshold(bottle_gray, 55, 255, cv2.THRESH_BINARY_INV)
    black_segment = cv2.bitwise_and(image, image, mask=bottle_threshold)
    # cv2_imshow(black_segment)

    total_pixels = image.shape[0] * image.shape[1]
    black_pixels = np.sum(bottle_threshold) // 255
    black_percentage = black_pixels / total_pixels
    # print(f'Percentual de preto: {black_percentage}')

    return black_percentage >= threshold

In [28]:
def red_detector(image, threshold=0.4):
    #label 40%
    #cover ?
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask_red1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    # cv2_imshow(mask_red)

    red_segment = cv2.bitwise_and(image, image, mask=mask_red)
    # cv2_imshow(red_segment)

    total_pixels = image.shape[0] * image.shape[1]
    red_pixels = np.sum(mask_red) // 255

    red_percentage = red_pixels / total_pixels
    # print(red_percentage)
    # print(f'Percentual de vermelho: {red_percentage}')
    return red_percentage >= threshold

In [40]:
def handler_bottom(img):
  height, width, _ = img.shape
  response = np.zeros(8)

  # Calculate the length of each part
  part_length = width // 3

  # Divide the image into three parts
  bottle1 = img[:, :part_length]
  bottle2 = img[:, part_length:2*part_length]
  bottle3 = img[:, 2*part_length:]

  cap1 = bottle1[:height//5, :]
  cap2 = bottle2[:height//5, :]
  cap3 = bottle3[:height//5, :]

  cover1 = bottle1[170:305,:]
  cover2 = bottle2[170:305,:]
  cover3 = bottle3[170:305,:]

  neck1 = bottle1[40:185,:]
  neck2 = bottle2[40:185,:]
  neck3 = bottle3[40:185,:]

  filled = black_detector(neck2)
  capped = red_detector(cap2, threshold=0.3)
  redCover =  red_detector(cover2)
  whiteCover = white_detector(cover2)
  covered = redCover or whiteCover
  unlabeled = black_detector(cover2, threshold=0.42)

  if filled or capped or covered:
    if not capped:
      response[2] = 1 
    if whiteCover:
      response[4] = 1 
    if unlabeled:
      response[6] = 1 
    # print(f'Tem tampa? {capped}')
    # print(f'Embalagem Vermelha? {redCover}')
    # print(f'Embalagem branca? {whiteCover}')
    # print(f'Sem embalagem? {unlabeled}')
    if not unlabeled:
      if not detect_misplaced_covers(cover2):
        response[5] = 1 
      # print(f'Embalagem posicionada corretamente? {detect_misplaced_covers(cover2)}')
    if filled:
      if bottom_level(neck2):
        # print('Nivel Alto')
        response[0] = 1
    else:
      # response[0] = 0
      response[1] = 1
      # print('Nivel Baixo')
  else:
    response[7] = 1
    # print('Sem garrafa')

  return response

In [43]:
for i in range(1, 78):
  image = cv2.imread(f'train_{i}.jpg')
  print(f'train_{i}.jpg')
  print(handler_bottom(image))
  # image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # plt.imshow(image_rgb)
  # plt.axis('off')  # Optional: Turn off axis labels
  # plt.show()
  # print('--------------------------------------')
  print()

train_1.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]

train_2.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]

train_3.jpg
[1. 0. 1. 0. 0. 1. 0. 0.]

train_4.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]

train_5.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]

train_6.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]

train_7.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]

train_8.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]

train_9.jpg
[0. 0. 0. 0. 0. 0. 0. 1.]

train_10.jpg
[1. 0. 0. 0. 0. 1. 0. 0.]

train_11.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]

train_12.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]

train_13.jpg
[0. 0. 0. 0. 0. 0. 0. 1.]

train_14.jpg
[0. 0. 0. 0. 0. 0. 1. 0.]

train_15.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]

train_16.jpg
[0. 0. 0. 0. 1. 0. 0. 0.]

train_17.jpg
[0. 1. 1. 0. 0. 0. 0. 0.]

train_18.jpg
[1. 0. 0. 0. 0. 0. 0. 0.]

train_19.jpg
[1. 0. 0. 0. 0. 1. 0. 0.]

train_20.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]

train_21.jpg
[1. 0. 0. 0. 0. 0. 0. 0.]

train_22.jpg
[0. 1. 0. 0. 0. 0. 0. 0.]

train_23.jpg
[1. 0. 1. 0. 0. 0. 1. 0.]

train_24.jpg
[1. 0. 0. 0. 0. 0. 0. 0.]

train_25.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]

train_26.

In [ ]:
sample = cv2.imread(f'train_10.jpg')
handler_bottom(sample)

Percentual de preto: 0.26790450928381965
Percentual de vermelho: 0.36857100014994754
Percentual de vermelho: 0.6479791395045632
Percentual de branco: 0.0950311458786035
Percentual de preto: 0.13197160654787773
Tem tampa? True
Embalagem Vermelha? True
Embalagem branca? False
Sem embalagem? False
Embalagem posicionada corretamente? False
Nivel Alto
